In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
from pylj import md, util, sample

warnings.filterwarnings('ignore')

# Atomistic simulation

The use of computers in chemistry is becoming more common as computers are increasing in power and the algorithms are becoming more accurate and efficient. Computer simulation of atomic and molecular species takes two "flavours":

- **Quantum calculations**: where approximations are made to the Schrödinger equation to allow for its calculation for a multi-electron system (more information about this can be found in Atkins & Friedman's *Molecular Quantum Mechanics*),
- **Classical simulations**: where model interactions between atoms are developed to allow chemical systems to be simulated,

This exercise will focus on the latter.

## Classical simulation

One of the most popular models used to simulate interparticle interactions is known as the **Lennard-Jones** function. This models the London dispersion interactions between atoms. Hopefully the London dispersion interactions are familiar as consisting of: 

- **van der Waals attraction**: the attraction between atoms that occurs as a result of the formation of instantaneous dipole formation,
- **Pauli's exclusion principle**: the repulsion which stops atoms overlapping as no two electrons can have the same quantum state. 

This means that the Lennard-Jones function is attractive when particles are close enough to induce dipole formation but **very** repulsive when the particles are too close. The Lennard-Jones function has the following form,

$$ E(r) = \frac{A}{r^{12}} - \frac{B}{r^6}, $$

where $E(r)$ is the potential energy at a given distance $r$, while $A$ and $B$ are parameters specific to the nature of the given interaction. In the cell below, write a function to calculate the energy of an interaction between two argon atoms for given range of distances using the Lennard-Jones function as the model.

In [ ]:
def lennard_jones(r, epsilon, sigma):
    return ◽◽◽

Then use this function to plot the potential energy from $r=3$ Å to $r=8$ Å, and discuss the shape and sign of this function in terms of the attractive and repulsive regimes, when the values of ε and σ are as follows.

| A/$\times10^{-134}$Jm$^{12}$ | B/$\times10^{-78}$Jm$^6$ |
|:------:|:------:|
| 1.36 | 9.27 |

In [ ]:
r = np.linspace( ◽◽◽, ◽◽◽, ◽◽◽ )
E = ◽◽◽

%matplotlib widget
plt.plot( ◽◽◽, ◽◽◽ )
plt.xlabel( ◽◽◽ )
plt.ylabel( ◽◽◽ )
plt.show()

This exercise will focus on the Lennard-Jones potential, however other functional forms exist to model other interactions between atoms, such as a harmonic well for bond lengths and angles and the Coulomb's law to model the interaction between charged particles. The total energy of interaction between two atoms arises as a sum of the different individual interactions. 

## Molecular dynamics

One possible method for classical simulation of a system is the use of **molecular dynamics**. This is where classical mechanics (developed by Isaac Newton) are leveraged to obtain dynamical (time-dependent) information about the system being modelled. 

Using classical mechanics we can determine the displacement, $x-x_0$, of some particle after a given time $\Delta t$, 

$$ x_{\Delta t} = x_0 + \mathbf{v}_0\Delta t + \frac{1}{2}\mathbf{a}_0\Delta t^2 $$, 

where, $\mathbf{v}_0$ is the velocity of the particle and $\mathbf{a}_0$ the acceleration. This means that in order to determine the trajectory of an atom (or a series of atoms) it is important that we understand it's acceleration. Which leads us to the consideration of Newton's second law of motion, 

$$ \mathbf{F}_0 = m\mathbf{a}_0, $$

where, $\mathbf{F}_0$ is the force on the particle and $m$ is the particles mass. To determine the force on the particle we use the correlation between the force, and the potential energy of the system, 

$$ \mathbf{F}_0 = -\frac{\text{d}E(r_0)}{\text{d}r_0}, $$

which is to say that the negative of the first derivative of the energy with respect to distance gives the force. 

Using this information, define a function for the force between a pair of argon atoms and plot this with respect to the distance.

In [ ]:
def lennard_jones_force(r, epsilon, sigma):
    return ◽◽◽

r = np.linspace( ◽◽◽, ◽◽◽, ◽◽◽ )
F = ◽◽◽

%matplotlib widget
plt.plot( ◽◽◽, ◽◽◽ )
plt.xlabel( ◽◽◽ )
plt.ylabel( ◽◽◽ )
plt.show()

Despite now having a method to find the acceleration on the particles, it is still not clear how the velocities can be found to iterates through equation 2. For this, we make us of another of the equations of motion, 

$$ \mathbf{v}_{\Delta t} = \mathbf{v}_0 + \frac{1}{2}(\mathbf{a}_{\Delta t} + \mathbf{a}_0)\Delta t, $$

where the acceleration it taken as the average of the acceleration before and after the move. These steps are the
essence of a **molecular dynamics integrator**.

## pylj 

In this exercise, we will use the Python library pylj as the 2 dimensional molecular dynamics engine to simulate our system. pylj consists of a series of functions to perform a molecular dynamics and Monte-Carlo simulations of argon particles interacting via a Lennard-Jones potential. Full documentation for all of the functions in pylj is available [here](http://pylj.readthedocs.io/en/latest/). The cell below will create a function which allows for the NVT molecular dynamics simulation to be performed, the NVT means that the number of atoms (N), volume of the system (V), and temperature of the system (T) will all be held constant. 

In [ ]:
def md_setup(number_of_particles, temperature, box_length):
    # Creates the visualisation environment
    %matplotlib widget
    # Initialise the system
    system = md.initialise(number_of_particles, temperature, box_length, 'square')
    # This sets the sampling class
    sample_system = sample.MaxBolt(system)
    return system, sample_system

def md_simulation(system, sample_system, temperature, number_of_steps, sampling_class):
    # Start at time 0
    system.time = 0
    # Begin the molecular dynamics loop
    for i in range(0, number_of_steps):
        # At each step, calculate the forces on each particle and get acceleration
        system.compute_force() 
        # Run the equations of motion integrator algorithm
        system.integrate(md.velocity_verlet)
        # Sample the thermodynamic and structural parameters of the system
        system = md.sample(system.particles, system.box_length, system.initial_particles, system)
        # Allow the system to interact with a heat bath
        system.heat_bath(temperature)
        # Iterate the time
        system.time += system.timestep_length
        system.step += 1
        # At a given frequency sample the positions and plot the RDF
        if system.step % 50 == 0:
            sample_system.update(system)
    if sampling_class == sample.RDF:
        sample_system.average()
    return system

The `md_simulation` function takes five variables:
- The number of particles
- The simulation temperature
- The simulation cell vector
- The number of steps
- The [sampling class](http://pylj.readthedocs.io/en/latest/sample.html) to be used. 

Initially we will simulate just four particles in a box, run the cell below and watch the simulation occur.

In [ ]:
system, sample_system = md_setup(4, 100., 100)

In [ ]:
simulation1 = md_simulation(system, sample_system, 100., 10000, sample.JustCell)

Notice that as the particles reach one side of the simulation cell, they will automatically reappear on the other side of the cell, this is due to the **periodic boundary condition**. In the cell below, discuss what the periodic boundary condition is and how is effects the simulation (include references). 

After running a simulation in pylj, it is possible to return information collected during the simulation, such as:
- pressure: `pressure_sample`
- temperature: `temperature_sample`
- force: `force_sample`
- energy: `energy_sample`

These are obtained as shown in the cell below.

In [ ]:
print(simulation1.pressure_sample)

## The effect of temperature

Set up a pylj simulation similar to that above, however instead of the `JustCell` sampling class use the `Energy` sampling class and with 40 atoms in the system and a box length of 40 Å. Now run the simulation a series of times with different values for the temperature, ranging from 10 K to 1000 K. In the cell below, describe how the system appears to change and the effect on the energy of the system as the temperature is varied.  

In [ ]:
system2, sample_system2 = md_setup(40, 10, 40)

In [ ]:
simulation2 = md_simulation(system2, sample_system2, 10, 10000, sample.Energy)

## Radial distribution function

The radial distribution function is a probability function, where the y-axis is the probability that an atom will be found at a given distance (x-axis) from the center of another atom. The radial distribution function of a given system can be accessed by envoking the `RDF` sampling class in pylj. Set the box length to 40 Å and the temperature to 100 K, now vary the number of particle in the simulation from 20 to 80 to 150 and discuss the resulting time-averaged radial distribution function that is given at the end of the simulations (investigate in the literature what information is often determined from the radial distribution function). 

In [ ]:
system3, sample_system3 = md_setup(80, 100, 40,)

In [ ]:
simulation3 = md_simulation( system3, sample_system3 , 1000, 10000, sample.RDF)

## Simulating an ideal gas

Argon at standard temperature and pressure (STP) is often used as an example of an ideal gas. In the cell below, discuss the requirements for a system to act ideally.

The density of argon at STP is 1.784 g/L, write a function to convert this from units of g/L to atoms/Å<sup>3</sup>. Then find the size of the box length that would be required simulate an argon gas at STP with pylj (remember that pylj performs 2-dimensional simulations and this must be accounted for).

In [ ]:
def units_conversion(density, atomic_mass):
    ◽◽◽

Perform a simulation such that the the density is that of argon at STP using the `Interactions` sampling cell, in the cell below discuss if argon at this density and temperature is a suitable model for an ideal gas (make sure to comment on the total force over the whole simulation). 

In [ ]:
system4, sample_system4 = md_setup(◽◽◽, 273.15, ◽◽◽)

In [ ]:
simulation4 = md_simulation( system4, sample_system4, 273.15, 10000, sample.Interactions)

The ideal gas law describes the behaviour of ideal gases across a range of temperatures, pressures and densities, 

$$ pV = Nk_BT, $$

where, $p$ is the pressure, $V$ is the volume, $N$ is the number of atoms, $k_B$ is the Boltzmann constant and $T$ is the temperature. This relationship can be proven from a multi particle partition function as shown [here](./first_principles.ipynb), abstract the proof given to show that the ideal gas law in two dimensions is as follows, 

$$ pA = Nk_BT, $$

where, $A$ is the area. 

Simulate the argon system at a series of densities, we recommend keeping the box length as 40 Å and the temperature at 273 K while varying the number of particles up to 100. Calculate the mean pressure measured throughout each simulation and plot this against the number of particles, also plot the ideal gas law on the same graph. Identify any deviations from the ideal gas law that are present and discuss the reasons. 

In [ ]:
system5, sample_system5 = md_setup(◽◽◽, 273.15, 40)

In [ ]:
simulation5 = md_simulation(system5, sample_system5, 273.15, 10000, sample.Interactions)
# print the mean pressure
print(simulation4.pressure_sample.mean())
# print the standard deviation in the mean pressure
print(simulation4.pressure_sample.std())

In [ ]:
%matplotlib inline
# Insert the plotting code here

At high densities, it is noted that systems with interactions between them (such as those modelled with the Lennard-Jones function) act in a non-ideal nature. This can be represented symbolically with the van der Waals equation, 

$$ p = \frac{k_BT}{v - b} - \frac{a}{v^2} $$

where, $v = A / N$, and $a$ and $b$ are constants determined by comparison with experimental data. 

Investigate the [`curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function by reading the documentation online, and use it to calculate values of $a$ and $b$ for the two dimensional interactions of the argon system (good starting values for $a$ and $b$ are $1\times10^{-40}$ and $1\times10^{-19}$ respectively).